In [80]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD, RMSprop, Adam
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras.callbacks import TensorBoard
from keras import backend as K

import numpy as np

# TODO
- more aggresive data augmentation
- more aggressive dropout
- use of L1 and L2 regularization (also known as "weight decay")
- fine-tuning one more convolutional block (alongside greater regularization)


In [68]:
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data/train'
validation_data_dir = 'data/valid'
test_data_dir = 'data/test'
nb_train_samples = 6363
nb_epochs = 15
batch_size = 16
nb_classes = 3

In [69]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [72]:
train_datagen = ImageDataGenerator(rescale=1. / 255)
itr = train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=nb_train_samples,
    class_mode='categorical')

Found 6363 images belonging to 3 classes.


In [73]:
X, y = itr.next()

In [74]:
print(X.shape)
print(y.shape)

(6363, 224, 224, 3)
(6363, 3)


In [75]:
pre_processed_X = preprocess_input(X)

In [76]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
print('Model loaded.')

Model loaded.


In [81]:
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)
predictions = Dense(nb_classes, activation='softmax', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(0.01))(x)

In [82]:
# this is the model we will train
# add your top layer block to your base model
model = Model(base_model.input, predictions)

In [83]:
def printAllLayers():
    for i, layer in enumerate(model.layers):
       print(i, layer.name)

In [84]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
printAllLayers()

0 input_2
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten_4
20 dense_3
21 dropout_2
22 dense_4


In [85]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:19]:
    layer.trainable = False

In [86]:
def printTrainableLayers():
    for i, layer in enumerate(model.layers):
       if layer.trainable == True: 
           print(i, layer.name)

In [87]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
printTrainableLayers()

19 flatten_4
20 dense_3
21 dropout_2
22 dense_4


In [88]:
# checkpoint
filepath="vgg16-top-weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [89]:
#histogram_freq=0, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list.append(tbCallBack)

In [ ]:
#opt = RMSprop(lr=0.1)
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#SGD(lr=1e-4, momentum=0.9)
#opt=SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Goal: < 0.96188 val_loss

In [ ]:
history = model.fit(pre_processed_X, y, batch_size=16, epochs=50, callbacks=callbacks_list, validation_split=0.3, shuffle=True, initial_epoch=0)

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_vgg16_top_with_dropout.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_vgg16_top_with_dropout.h5")
print("Saved model to disk")

In [47]:
# load json and create model
json_file = open("model_vgg16_with_dropout.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("vgg16-weights-improvement-06-0.98.hdf5")

In [48]:
printAllLayers()

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten_1
20 dense_1
21 dropout_1
22 dense_2


In [49]:
printTrainableLayers()

19 flatten_1
20 dense_1
21 dropout_1
22 dense_2


In [50]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers[15:]:
    layer.trainable = True    

In [51]:
printTrainableLayers()

15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten_1
20 dense_1
21 dropout_1
22 dense_2


In [52]:
from keras.optimizers import SGD
opt=SGD(lr=1e-4, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy')

In [53]:
history = model.fit(zero_centered_X, y, batch_size=16, epochs=50, callbacks=callbacks_list, validation_split=0.3, shuffle=True, initial_epoch=0)

Train on 4143 samples, validate on 1776 samples
Epoch 1/50
4143/4143 [==============================] - 322s - loss: 0.9188 - val_loss: 0.9790
Epoch 2/50
4143/4143 [==============================] - 304s - loss: 0.9075 - val_loss: 0.9776
Epoch 3/50
4143/4143 [==============================] - 190s - loss: 0.8853 - val_loss: 0.9786
Epoch 4/50
4143/4143 [==============================] - 79s - loss: 0.8783 - val_loss: 0.9724
Epoch 5/50
4143/4143 [==============================] - 78s - loss: 0.8575 - val_loss: 0.9793
Epoch 6/50
4143/4143 [==============================] - 78s - loss: 0.8421 - val_loss: 0.9923
Epoch 7/50
4143/4143 [==============================] - 78s - loss: 0.8395 - val_loss: 0.9905
Epoch 8/50
4143/4143 [==============================] - 77s - loss: 0.8224 - val_loss: 0.9830
Epoch 9/50
4143/4143 [==============================] - 77s - loss: 0.8191 - val_loss: 0.9939
Epoch 10/50
4143/4143 [==============================] - 77s - loss: 0.8015 - val_loss: 0.9918
Epoch 11

KeyboardInterrupt: 

In [54]:
# serialize model to JSON
model_json = model.to_json()
with open("model_vgg16_with_dropout.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_vgg162_with_dropout.h5")
print("Saved model to disk")

Saved model to disk


In [55]:
# load json and create model
json_file = open("model_vgg16_with_dropout.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("vgg16-weights-improvement-03-0.97.hdf5")

In [57]:
printTrainableLayers()

15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten_1
20 dense_1
21 dropout_1
22 dense_2


# Predict:

In [61]:
# Read Data
test_datagen = ImageDataGenerator(rescale=1. / 255)
itr = test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=512,
                                                  shuffle=False)

Found 512 images belonging to 1 classes.


In [62]:
test_data = itr.next()

In [63]:
pre_processed_test_data = preprocess_input(test_data)

TypeError: tuple indices must be integers or slices, not tuple

In [ ]:
zero_centered_test_data = pre_processed_test_data